MODEL TRAINING

In [1]:
import pandas as pd

In [2]:
data = pd.read_csv("https://raw.githubusercontent.com/krishnaik06/FSDSRegression/main/notebooks/data/gemstone.csv")

In [3]:
data.head()

,id,carat,cut,color,clarity,depth,table,x,y,z,price
0,0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55,13619
1,1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05,13387
2,2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50,2772
3,3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71,666
4,4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77,14453


In [4]:
data = data.drop(labels=["id"], axis=1)

In [5]:
data.head()

,carat,cut,color,clarity,depth,table,x,y,z,price
0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55,13619
1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05,13387
2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50,2772
3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71,666
4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77,14453


In [6]:
#independent and dependent features

X = data.drop(labels=["price"], axis=1)
Y = data[["price"]]

In [7]:
#define which column should be ordinal encoded and which one should be scaled

categorical_cols = X.select_dtypes(include="object").columns.tolist()
numerical_cols = X.select_dtypes(exclude="object").columns.tolist()

In [8]:
categorical_cols, numerical_cols

(['cut', 'color', 'clarity'], ['carat', 'depth', 'table', 'x', 'y', 'z'])

In [9]:
#define the custom ranking for each ordinal variable

cut_categories = ["Fair", "Good", "Very Good", "Premium", "Ideal"]
color_categories = ["D", "E", "F", "G", "H", "I", "J"]
clarity_categories = ["I1", "SI2", "SI1", "VS2", "VS1", "VVS2", "VVS1", "IF"]

In [10]:
from sklearn.impute import SimpleImputer  #Handling missing values
from sklearn.preprocessing import StandardScaler #Handling feature scaling
from sklearn.preprocessing import OrdinalEncoder #Ordinal Encoding
#pipelines
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [11]:
#numerical pipeline

num_pipeline = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler())
    ]
)

#categorical pipeline

cat_pipeline = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("ordinalencoder", OrdinalEncoder(categories=[cut_categories, color_categories, clarity_categories])),
        ("Scaler", StandardScaler())
    ]
)

preprocessor = ColumnTransformer(
    [
        ("num_pipeline", num_pipeline, numerical_cols),
        ("cat_pipeline", cat_pipeline, categorical_cols)
    ]
)

In [12]:
#Train test split

from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.30, random_state=30)

In [13]:
preprocessor.fit_transform(X_train)

array([[-0.97543926, -0.84960654, -0.12153081, ...,  0.87407553,
         1.52872212,  1.35273128],
       [ 0.2351953 ,  1.83363716, -0.12153081, ..., -2.14455824,
        -0.93507064, -0.64678628],
       [ 0.49461699,  0.81585507,  0.39980029, ..., -0.13213573,
         0.29682574,  0.68622543],
       ...,
       [ 0.45138004,  1.55606023, -0.6428619 , ..., -2.14455824,
         0.29682574, -0.64678628],
       [ 0.66756478, -1.77486298,  1.44246248, ...,  0.87407553,
         0.29682574,  0.68622543],
       [ 0.25681377,  0.81585507, -0.12153081, ...,  0.87407553,
         0.29682574, -0.64678628]])

In [14]:
X_train = pd.DataFrame(preprocessor.fit_transform(X_train), columns=numerical_cols+categorical_cols)


In [15]:
X_train

,carat,depth,table,x,y,z,cut,color,clarity
0,-0.975439,-0.849607,-0.121531,-1.042757,-1.080970,-1.123150,0.874076,1.528722,1.352731
1,0.235195,1.833637,-0.121531,0.318447,0.279859,0.485354,-2.144558,-0.935071,-0.646786
2,0.494617,0.815855,0.399800,0.570855,0.606458,0.673737,-0.132136,0.296826,0.686225
3,-1.018676,0.260701,0.921131,-1.214034,-1.244270,-1.195605,-0.132136,0.296826,0.019720
4,-0.953821,-0.664555,-0.642862,-1.069801,-1.044681,-1.094168,0.874076,2.144670,1.352731
...,...,...,...,...,...,...,...,...,...
135496,-1.040295,-0.016876,-0.642862,-1.268122,-1.244270,-1.239078,0.874076,-0.935071,-0.646786
135497,0.991842,0.168176,-0.642862,1.048629,1.114501,1.079486,0.874076,0.296826,-1.313292
135498,0.451380,1.556060,-0.642862,0.516768,0.588314,0.702719,-2.144558,0.296826,-0.646786
135499,0.667565,-1.774863,1.442462,0.868337,0.951202,0.688228,0.874076,0.296826,0.686225


In [16]:
X_test = pd.DataFrame(preprocessor.transform(X_test), columns=numerical_cols+categorical_cols)

In [17]:
X_test

,carat,depth,table,x,y,z,cut,color,clarity
0,-0.564688,-0.942132,-0.642862,-0.429765,-0.464061,-0.500036,-0.132136,-0.935071,0.019720
1,-0.175556,1.000906,-0.121531,-0.042137,-0.028595,0.036132,-1.138347,0.912774,-0.646786
2,-1.061913,0.260701,-0.121531,-1.304180,-1.298703,-1.268060,0.874076,0.912774,2.685743
3,0.970223,-0.201927,1.963794,1.048629,0.996563,0.978049,-0.132136,0.296826,0.019720
4,-0.932202,-1.312235,0.399800,-1.006699,-0.990248,-1.065186,-0.132136,-0.935071,0.686225
...,...,...,...,...,...,...,...,...,...
58067,1.013460,1.185958,-0.642862,1.003556,1.041924,1.151941,-1.138347,0.912774,0.019720
58068,-0.997058,0.260701,-1.164193,-1.141917,-1.126331,-1.108659,0.874076,-0.319122,2.019237
58069,-0.197174,-3.347799,1.442462,0.102096,0.071199,-0.224706,-0.132136,2.144670,0.019720
58070,-0.824110,-0.201927,-0.121531,-0.853450,-0.881382,-0.876803,0.874076,0.296826,-0.646786


In [18]:
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [19]:
regression = LinearRegression()

In [20]:
regression.fit(X_train, Y_train)

LinearRegression()

In [21]:
regression.coef_

array([[ 6433.66003594,  -132.75843566,   -70.42922179, -1720.30971463,
         -499.29302619,   -63.39317848,    72.44537247,  -460.41604642,
          650.76431652]])

In [22]:
regression.intercept_

array([3970.76628955])

In [23]:
y_predict = regression.predict(X_test)

In [24]:
mean_squared_error(Y_test, y_predict)

1028002.7598132562

In [25]:
r2_score(Y_test, y_predict)

0.9368908248567511

In [26]:
import numpy as np

In [29]:
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    R2_score = r2_score(true, predicted)
    return mae, rmse, R2_score

In [34]:
##train multiple models

models = {
    "LinearRegression":LinearRegression(),
    "Lasso":Lasso(),
    "Ridge":Ridge(),
    "ElasticNet":ElasticNet()
}

model_list = []
r2_list = []

for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train, Y_train)


    #make predictions
    y_pred = model.predict(X_test)

    mae, rmse, R2_score = evaluate_model(Y_test, y_pred)
    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])
    r2_list.append(R2_score)
    

    print("Model Training Performance")
    print("RMSE : ", rmse)
    print("MAE : ", mae)
    print(f"R2_score : {R2_score*100}%")
    print("="*30)
    print("\n")

LinearRegression
Model Training Performance
RMSE :  1013.9047094344005
MAE :  674.0255115796832
R2_score : 93.68908248567512%


Lasso
Model Training Performance
RMSE :  1013.8784226767013
MAE :  675.0716923362158
R2_score : 93.68940971841704%


Ridge
Model Training Performance
RMSE :  1013.9059272771653
MAE :  674.0555800798319
R2_score : 93.68906732505937%


ElasticNet
Model Training Performance
RMSE :  1533.4162456064046
MAE :  1060.7368759154729
R2_score : 85.56494831165182%




In [1]:
import pandas as pd
pd.read_csv("gemstone.csv")

,Unnamed: 0,index,id,carat,cut,color,clarity,depth,table,x,y,z,price
0,0,0,0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55,13619
1,1,1,1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05,13387
2,2,2,2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50,2772
3,3,3,3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71,666
4,4,4,4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77,14453
...,...,...,...,...,...,...,...,...,...,...,...,...,...
193568,193568,193568,193568,0.31,Ideal,D,VVS2,61.1,56.0,4.35,4.39,2.67,1130
193569,193569,193569,193569,0.70,Premium,G,VVS2,60.3,58.0,5.75,5.77,3.47,2874
193570,193570,193570,193570,0.73,Very Good,F,SI1,63.1,57.0,5.72,5.75,3.62,3036
193571,193571,193571,193571,0.34,Very Good,D,SI1,62.9,55.0,4.45,4.49,2.81,681
